# 6 - Functionalizing Ligands 

Beyond simple ligands searched from SMILES, we want to dig into functionalizations of ligands!

In this tutorial we will look at the ligand functionalization routines in Architector, covering:

**(A)** Viewing default functional groups present in architector by name!

**(B)** Identifying functionalization sites on ligands.

**(C)** Adding both single and multiple functionalizations to a single ligand.

In [1]:
# First, import useful packages again
from architector import (build_complex, # Build routine
                         view_structures, # Visualization
                         smiles2Atoms, # Smiles to ASE Atoms
                         get_obmol_smiles, # Coversion to OBmol routine for editing ligand smiles
                         get_smiles_obmol, # Convert OBmol to SMILES string
                         convert_obmol_ase) # Conversion of OBmol molecele to ASE atoms for visualization
from architector.io_ptable import functional_groups_dict # Default functional groups dictionary
import copy

# For (A), Some Functional groups are included by default. Otherwise they can be input as SMILES strings!

Here, I will visualize all functional groups present by default in Architector. We will write a quick function for placing the functional groups onto a Benzene ring for clarity!

In [2]:
def view_functional_group(name, base_group='c1ccccc1', base_inds=[5]):
    """
    function to visualize functional groups on a base_group. 
    
    Inputs:
    name : str
        Name of the functional group to put on the base_group.
    base_group : str, optional
        SMILES string of the organic group to functionalize! by default benzene.
    base_inds : list(Int) 
        list of the indices of the base_smiles to add the functional group to.
    """
    fgs = [{'functional_group':name,'smiles_inds':base_inds}] # Construct functional group dictionary list.
    OBmol = get_obmol_smiles(base_group, functionalizations=fgs) # Perform functionalization in Opebabel
    new_smiles = get_smiles_obmol(OBmol) # Get the new smiles for this OBmol molecule
    ase_atoms = convert_obmol_ase(OBmol) # Convert the molecule to ASE atoms for viewing
    # Print out the functional group name and edited SMILES string
    print('Name: {}\t Functionalized_Smiles: {}'.format(name,new_smiles))  
    view_structures(ase_atoms) # View the structures.

In [3]:
'OP(O)O'

'OP(O)O'

Now, we iterate through the default functional group dictionary, visualizing what each group looks like on benzene!

In [4]:
for key,val in functional_groups_dict.items():
    view_functional_group(key)

Name: methyl	 Functionalized_Smiles: c1ccccc1C


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Name: ethyl	 Functionalized_Smiles: c1ccccc1CC


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Name: phenyl	 Functionalized_Smiles: c1ccccc1C1=CC=CC=C1


...

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Name: 2-hydroxypyradine	 Functionalized_Smiles: c1ccccc1C1=CC=CC(=N1)O


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Name: 2-methylbenzoic_acid	 Functionalized_Smiles: c1ccccc1C1=C(C=CC(=C1)C)C(=O)O


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

# For (B), We can also functionalize ligands during complex construction! But we need to know where to functionalize them.

Fort this example we will be functionalizing bipyradine (bipy) when bound to Fe in a couple different ways to highlight this functionality.

To start, we need the base bipy SMILES and coordinating atoms:

In [5]:
# From online:
bipy_smiles = 'n1ccccc1-c2ccccn2'
metal = 'Fe' # Initilize metal

Now, we can view the structures. 

### Instead of just looking for the coordinating atoms, we can also identify functionalization sites:

In [6]:
bipy_atoms = smiles2Atoms(bipy_smiles)
view_structures(bipy_atoms,labelinds=True,w=500,h=500) 

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

As before, the coordination sites are the Nitrogens (blue) with indices 0 and 11, and that bipy will be a "bi_cis" ligand.

### However, if we want to put two functional groups at both of the positions "para" to the Nitrogens, these sites are the Carbons with indices 3 and 8!

In [7]:
bipy_coordList = [0,11]
bipy_ligType = 'bi_cis'
para_smiles_inds = [3,8]

## For (C), Now we can make a functional group list of dictionaries for the bipy during complex construction! 

Here, let's just use the chloro functionalization from above for simplicity!

In [8]:
para_functional_groups = [{'functional_group':'chloro','smiles_inds':para_smiles_inds}]

Now we have enough to do a functionalized complex construction with Fe!

In [9]:
# We now have what we need to make an Fe-Bipy complex with functionalizations
lig_dict = {'smiles':bipy_smiles,
            'coordList':bipy_coordList,
            'ligType':bipy_ligType,
            'functionalizations':para_functional_groups}

inputDict = {'core':{
    'metal':'Fe',
    'coreType':'octahedral'  # Just making octahedral complexes for simplicity
        },
    'ligands':[lig_dict], # Add in the ligands dictionary
    'parameters':{
        'assemble_method':'GFN-FF', # Switch to GFN-FF for faster assembly, 
        'relax':False, # Turn of relaxation for non-optimized structures
        'fill_ligand':0
                 }
    }

In [10]:
out = build_complex(inputDict)


          ==================== Thresholds ====================
          CN  :   150.00000
          rep :   500.00000
          disp:  2500.00000
          HB1 :   250.00000
          HB2 :   450.00000

          Pauling EN used:
          Z : 1  EN :  2.20
          Z : 6  EN :  2.55
          Z : 7  EN :  3.04
          Z :17  EN :  3.16
          Z :26  EN :  1.83
          electric field strengths (au): 0.000

           ------------------------------------------------- 
          |           Force Field Initialization            |
           ------------------------------------------------- 

          distances ...
          ----------------------------------------
          generating topology and atomic info file ...
          pair mat ...
          computing topology distances matrix with Floyd-Warshall algo ...
          making topology EEQ charges ...
          #fragments for EEQ constrain: 1
          ----------------------------------------
          generating topology 

          # BATM   200
          # H in HB   6
          doing iterative Hueckel for 1 subsystem(s) ...

  atom   neighbors  erfCN metchar sp-hybrid imet pi  qest     coordinates
    1  Fe      2    1.97   0.32         2    2    0   0.408    0.000000    0.000000    0.000000
    2  N       3    2.77   0.00         2    0    1  -0.027    3.687758   -0.000000    0.067935
    3  C       3    2.84   0.00         2    0    1   0.051    5.390722   -0.000014   -1.830568
    4  C       3    2.84   0.00         2    0    1   0.041    7.962743   -0.000023   -1.450833
    5  C       3    2.83   0.00         2    0    1   0.057    8.798012   -0.000024    1.027213
    6  C       3    2.84   0.00         2    0    1   0.026    7.089035   -0.000017    3.030016
    7  C       3    2.88   0.00         2    0    1   0.084    4.508016   -0.000011    2.509879
    8  C       3    2.88   0.00         2    0    1   0.084    2.509878    0.000008    4.508007
    9  C       3    2.84   0.00         2    0    1  


  atom   neighbors  erfCN metchar sp-hybrid imet pi  qest     coordinates
    1  Fe      4    3.50   0.18         3    2    0   0.324    0.000000    0.000000    0.000000
    2  N       3    2.77   0.00         2    0    1  -0.121    3.696193   -0.000000    0.176343
    3  C       3    2.84   0.00         2    0    1   0.030    5.365298   -0.000011   -1.751620
    4  C       3    2.84   0.00         2    0    1   0.011    7.945429    0.000003   -1.417417
    5  C       3    2.83   0.00         2    0    1   0.048    8.828820   -0.000005    1.045532
    6  C       3    2.84   0.00         2    0    1   0.003    7.158586   -0.000011    3.078695
    7  C       3    2.87   0.00         2    0    1   0.063    4.574325    0.000001    2.592257
    8  C       3    2.87   0.00         2    0    1   0.063    2.592255   -0.000007    4.574333
    9  C       3    2.84   0.00         2    0    1   0.003    3.078704   -0.000013    7.158574
   10  C       3    2.83   0.00         2    0    1   0.048  


          ==================== Thresholds ====================
          CN  :   150.00000
          rep :   500.00000
          disp:  2500.00000
          HB1 :   250.00000
          HB2 :   450.00000

          Pauling EN used:
          Z : 1  EN :  2.20
          Z : 6  EN :  2.55
          Z : 7  EN :  3.04
          Z :17  EN :  3.16
          Z :26  EN :  1.83
          electric field strengths (au): 0.000

           ------------------------------------------------- 
          |           Force Field Initialization            |
           ------------------------------------------------- 

          distances ...
          ----------------------------------------
          generating topology and atomic info file ...
          pair mat ...
          computing topology distances matrix with Floyd-Warshall algo ...
          making topology EEQ charges ...
          #fragments for EEQ constrain: 1
          ----------------------------------------
          generating topology 


          ==================== Thresholds ====================
          CN  :   150.00000
          rep :   500.00000
          disp:  2500.00000
          HB1 :   250.00000
          HB2 :   450.00000

          Pauling EN used:
          Z : 1  EN :  2.20
          Z : 6  EN :  2.55
          Z : 7  EN :  3.04
          Z :17  EN :  3.16
          Z :26  EN :  1.83
          electric field strengths (au): 0.000

           ------------------------------------------------- 
          |           Force Field Initialization            |
           ------------------------------------------------- 

          distances ...
          ----------------------------------------
          generating topology and atomic info file ...
          pair mat ...
          computing topology distances matrix with Floyd-Warshall algo ...
          making topology EEQ charges ...
          #fragments for EEQ constrain: 1
          ----------------------------------------
          generating topology 


  atom   neighbors  erfCN metchar sp-hybrid imet pi  qest     coordinates
    1  Fe      6    4.25   0.05         0    2    0   0.328    0.000000    0.000000    0.000000
    2  N       3    2.77   0.00         2    0    1  -0.149    3.696193   -0.000000    0.176343
    3  C       3    2.84   0.00         2    0    1   0.023    5.365298   -0.000011   -1.751620
    4  C       3    2.84   0.00         2    0    1  -0.000    7.945429    0.000003   -1.417417
    5  C       3    2.83   0.00         2    0    1   0.045    8.828820   -0.000005    1.045532
    6  C       3    2.84   0.00         2    0    1  -0.005    7.158586   -0.000011    3.078695
    7  C       3    2.87   0.00         2    0    1   0.055    4.574325    0.000001    2.592257
    8  C       3    2.87   0.00         2    0    1   0.055    2.592255   -0.000007    4.574333
    9  C       3    2.84   0.00         2    0    1  -0.005    3.078704   -0.000013    7.158574
   10  C       3    2.83   0.00         2    0    1   0.045  


          ==================== Thresholds ====================
          CN  :   150.00000
          rep :   500.00000
          disp:  2500.00000
          HB1 :   250.00000
          HB2 :   450.00000

          Pauling EN used:
          Z : 1  EN :  2.20
          Z : 6  EN :  2.55
          Z : 7  EN :  3.04
          Z :17  EN :  3.16
          Z :26  EN :  1.83
          electric field strengths (au): 0.000

           ------------------------------------------------- 
          |           Force Field Initialization            |
           ------------------------------------------------- 

          distances ...
          ----------------------------------------
          generating topology and atomic info file ...
          pair mat ...
          computing topology distances matrix with Floyd-Warshall algo ...
          making topology EEQ charges ...
          #fragments for EEQ constrain: 1
          ----------------------------------------
          generating topology 

In [11]:
view_structures(out,w=500,h=500)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Looks like a chloro-functionalized bipy at the para-positions!

### Beyond this, we can create more than one type of functionalization

Here, let's put Chloros at the para-positions and bromos at the ortho-positions to the coordinating atoms.

The ortho positions correspond to Carbon indices 2 and 9 above

In [12]:
multi_fgs = [{'functional_group':'chloro','smiles_inds':[8,3]}, # Chloro at para positions
                     {'functional_group':'bromo','smiles_inds':[2,9]}] # Bromo at ortho-positions

new_inputDict = copy.deepcopy(inputDict) # Copy inputDict

new_inputDict['ligands'][0]['functionalizations'] = multi_fgs # Change bipy functionalization!
del new_inputDict['parameters']['fill_ligand'] # Remove fill ligand -> Default to H2O

In [13]:
out1 = build_complex(new_inputDict)


          ==================== Thresholds ====================
          CN  :   150.00000
          rep :   500.00000
          disp:  2500.00000
          HB1 :   250.00000
          HB2 :   450.00000

          Pauling EN used:
          Z : 1  EN :  2.20
          Z : 6  EN :  2.55
          Z : 7  EN :  3.04
          Z :17  EN :  3.16
          Z :26  EN :  1.83
          Z :35  EN :  2.96
          electric field strengths (au): 0.000

           ------------------------------------------------- 
          |           Force Field Initialization            |
           ------------------------------------------------- 

          distances ...
          ----------------------------------------
          generating topology and atomic info file ...
          pair mat ...
          computing topology distances matrix with Floyd-Warshall algo ...
          making topology EEQ charges ...
          #fragments for EEQ constrain: 1
          ----------------------------------------
  


          ==================== Thresholds ====================
          CN  :   150.00000
          rep :   500.00000
          disp:  2500.00000
          HB1 :   250.00000
          HB2 :   450.00000

          Pauling EN used:
          Z : 1  EN :  2.20
          Z : 6  EN :  2.55
          Z : 7  EN :  3.04
          Z :17  EN :  3.16
          Z :26  EN :  1.83
          Z :35  EN :  2.96
          electric field strengths (au): 0.000

           ------------------------------------------------- 
          |           Force Field Initialization            |
           ------------------------------------------------- 

          distances ...
          ----------------------------------------
          generating topology and atomic info file ...
          pair mat ...
          computing topology distances matrix with Floyd-Warshall algo ...
          making topology EEQ charges ...
          #fragments for EEQ constrain: 1
          ----------------------------------------
  


  atom   neighbors  erfCN metchar sp-hybrid imet pi  qest     coordinates
    1  Fe      4    3.42   0.14         3    2    0   0.378    0.000000    0.000000    0.000000
    2  N       3    2.77   0.00         2    0    1  -0.081    0.173315   -0.000000    3.689033
    3  C       3    2.92   0.00         2    0    1   0.033   -1.757673    0.000014    5.366608
    4  C       3    2.83   0.00         2    0    1   0.046   -1.433900    0.000017    7.955810
    5  C       3    2.86   0.00         2    0    1   0.043    1.036137    0.000014    8.848531
    6  C       3    2.85   0.00         2    0    1   0.011    3.064277    0.000009    7.153796
    7  C       3    2.87   0.00         2    0    1   0.072    2.585115   -0.000002    4.567130
    8  C       3    2.87   0.00         2    0    1   0.072    4.567148    0.000005    2.585132
    9  C       3    2.85   0.00         2    0    1   0.011    7.153806   -0.000009    3.064348
   10  C       3    2.86   0.00         2    0    1   0.043  


          ==================== Thresholds ====================
          CN  :   150.00000
          rep :   500.00000
          disp:  2500.00000
          HB1 :   250.00000
          HB2 :   450.00000

          Pauling EN used:
          Z : 1  EN :  2.20
          Z : 6  EN :  2.55
          Z : 7  EN :  3.04
          Z : 8  EN :  3.44
          Z :17  EN :  3.16
          Z :26  EN :  1.83
          Z :35  EN :  2.96
          electric field strengths (au): 0.000

           ------------------------------------------------- 
          |           Force Field Initialization            |
           ------------------------------------------------- 

          distances ...
          ----------------------------------------
          generating topology and atomic info file ...
          pair mat ...
          computing topology distances matrix with Floyd-Warshall algo ...
          making topology EEQ charges ...
          #fragments for EEQ constrain: 1
          ---------------

Now, we have a water co-coordinated bi-functionalized Fe-Bipy complex!

In [14]:
view_structures(out1,w=500,h=500)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Finally, we can make structures with both functionalized and un-functionalized ligands simultaneously.

Also, functional groups can be defined by SMILES strings. Here we introduce a butane functional group with a tail tricholor carbon end.

In [15]:
functional_groups = [{'functional_group':'CCCC(Cl)(Cl)Cl','smiles_inds':[8,3]}]

lig_dict = {'smiles':bipy_smiles,'coordList':bipy_coordList,'ligType':bipy_ligType, # Functionalized bipy
            'functionalizations':functional_groups}
lig_dict2 = {'smiles':bipy_smiles,'coordList':bipy_coordList,'ligType':bipy_ligType} # Unfunctionalized bipy

last_inputDict = {
    'core':{
        'metal':'Fe',
        'coreType':'octahedral'
    },
    'ligands':[lig_dict,lig_dict2,lig_dict2],
    'parameters':{
        'assemble_method':'GFN-FF',
        'relax':False
    }
}
last_inputDict

{'core': {'metal': 'Fe', 'coreType': 'octahedral'},
 'ligands': [{'smiles': 'n1ccccc1-c2ccccn2',
   'coordList': [0, 11],
   'ligType': 'bi_cis',
   'functionalizations': [{'functional_group': 'CCCC(Cl)(Cl)Cl',
     'smiles_inds': [8, 3]}]},
  {'smiles': 'n1ccccc1-c2ccccn2', 'coordList': [0, 11], 'ligType': 'bi_cis'},
  {'smiles': 'n1ccccc1-c2ccccn2', 'coordList': [0, 11], 'ligType': 'bi_cis'}],
 'parameters': {'assemble_method': 'GFN-FF', 'relax': False}}

In [16]:
out = build_complex(last_inputDict)

DETERMINING SYMMETRIES.
Total valid symmetries for core octahedral:  1
GENERATING CONFORMATIONS for n1ccccc1-c2ccccn2
CONFORMERS GENERATED for n1ccccc1-c2ccccn2
GENERATING CONFORMATIONS for n1ccc(cc1c1cc(ccn1)CCCC(Cl)(Cl)Cl)CCCC(Cl)(Cl)Cl
CONFORMERS GENERATED for n1ccc(cc1c1cc(ccn1)CCCC(Cl)(Cl)Cl)CCCC(Cl)(Cl)Cl
ASSEMBLING COMPLEX
LIGAND: n1ccccc1-c2ccccn2
FINDING CORRECT CONFORMER

          ==================== Thresholds ====================
          CN  :   150.00000
          rep :   500.00000
          disp:  2500.00000
          HB1 :   250.00000
          HB2 :   450.00000

          Pauling EN used:
          Z : 1  EN :  2.20
          Z : 6  EN :  2.55
          Z : 7  EN :  3.04
          Z :26  EN :  1.83
          electric field strengths (au): 0.000

           ------------------------------------------------- 
          |           Force Field Initialization            |
           ------------------------------------------------- 

          distances ...
          ---

          pair mat ...
          computing topology distances matrix with Floyd-Warshall algo ...
          making topology EEQ charges ...
          #fragments for EEQ constrain: 1
          ----------------------------------------
          generating topology and atomic info file ...
          pair mat ...
          computing topology distances matrix with Floyd-Warshall algo ...
          making topology EEQ charges ...
          #fragments for EEQ constrain: 1
          rings ...
          # BATM   1236
          # H in HB   22
          doing iterative Hueckel for 3 subsystem(s) ...

  atom   neighbors  erfCN metchar sp-hybrid imet pi  qest     coordinates
    1  Fe      6    4.24   0.05         0    2    0   0.357    0.000000    0.000000    0.000000
    2  N       3    2.77   0.00         2    0    1  -0.173    0.105437    0.000000    3.716091
    3  C       3    2.84   0.00         2    0    1   0.009   -1.780953    0.000010    5.433214
    4  C       3    2.80   0.00         2

          # BATM   1236
          # H in HB   22
          doing iterative Hueckel for 3 subsystem(s) ...

  atom   neighbors  erfCN metchar sp-hybrid imet pi  qest     coordinates
    1  Fe      6    4.24   0.05         0    2    0   0.357    0.000000    0.000000    0.000000
    2  N       3    2.77   0.00         2    0    1  -0.173    0.105437    0.000000    3.716091
    3  C       3    2.84   0.00         2    0    1   0.009   -1.780953    0.000010    5.433214
    4  C       3    2.80   0.00         2    0    1  -0.026   -1.392127    0.000013    8.012495
    5  C       3    2.82   0.00         2    0    1  -0.028    1.081373   -0.000014    8.874432
    6  C       3    2.81   0.00         2    0    1  -0.024    3.064750   -0.000011    7.132966
    7  C       3    2.87   0.00         2    0    1   0.034    2.544778   -0.000016    4.541293
    8  C       3    2.87   0.00         2    0    1   0.034    4.541295   -0.000018    2.544785
    9  C       3    2.81   0.00         2    0    1

Complex class generated:  True
ComplexSanity:  True


In [17]:
view_structures(out,w=500,h=500)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Looks great! Now we have covered functionalizations as well.

# Conclusions

In this tutorial we covered:

**(A)** Viewing default functional groups present in architector by name!

**(B)** Identifying functionalization sites on ligands.

**(C)** Adding both single and multiple functionalizations to a single ligand.